# Подготовка к анализу

## Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer
from scipy.stats import spearmanr
%matplotlib inline

ALPHA = 0.05

## Загрузка данных

Загрузим данные из файла "heart.csv" с датасетом, также выведем информацию о данных и посмотрим первые 5 наблюдений из датасета

In [4]:
data = pd.read_csv('heart.csv')
X = data.drop(columns=['target'])
y = data['target']
data_info = data.info()
data_head = data.head()

data_info, data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   age       303 non-null    int64
 1   sex       303 non-null    int64
 2   cp        303 non-null    int64
 3   trestbps  303 non-null    int64
 4   chol      303 non-null    int64
 5   fbs       303 non-null    int64
 6   restecg   303 non-null    int64
 7   thalach   303 non-null    int64
 8   exang     303 non-null    int64
 9   target    303 non-null    int64
dtypes: int64(10)
memory usage: 23.8 KB


(None,
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  target
 0   63    1   3       145   233    1        0      150      0       1
 1   37    1   2       130   250    0        1      187      0       1
 2   41    0   1       130   204    0        0      172      0       1
 3   56    1   1       120   236    0        1      178      0       1
 4   57    0   0       120   354    0        1      163      1       1)

## Описание данных


* **age** — возраст пациента
* **sex** — пол пациента (1 = мужчина, 0 = женщина)
* **cp** — тип боли в груди (1 = типичная стенокардия, 2 = атипичная стенокардия, 3 = другой вид боли, 4 = нет боли)
* **trestbps** — артериальное давление в состоянии покоя (мм ртутного столба, на момент госпитализации)
* **chol** — уровень холестерола (мг/дл)
* **fbs** — уровень сахара крови натощак выше 120 мг/дл (1 = да, 0 = нет)
* **restecg** — результат ЭКГ в состоянии покоя (0 = нормальный, 1 = абнормальный, 2 = признаки гипертрофии желудочка)
* **thalach** — максимальная зафиксированная частота сердцебиения
* **exang** — стенокардия в результате физической нагружки (1 = да, 0 = нет)
* **target** — наличие сердечно-сосудистого заболевания (1 = да, 0 = нет)

# Дескриптивный анализ

[Был проделан в файле lab1](lab1.ipynb)

# Стандартизация переменных


Стандартизация — это процесс приведения числовых признаков (фичей) к единому масштабу. Обычно это делается так, чтобы все переменные имели среднее значение 0 и стандартное отклонение 1. Это достигается путем вычисления z-оценки для каждого значения:  
$$
z = \frac{x - \mu}{\sigma}
$$
Где:
* $x$ — текущее значение признака,
* $\mu$ — среднее значение признака,
* $\sigma$ — стандартное отклонение признака.

## Нужно ли выполнять стандартизацию?

In [11]:
for column in X.columns:
    print(column)
    print(f"Count of uniqeu: {len(X[column].unique())}\nValues: {X[column].unique()}")
    print(f'Min = {X[column].min()} Max = {X[column].max()}')
    print('-' * 50)

age
Count of uniqeu: 41
Values: [63 37 41 56 57 44 52 54 48 49 64 58 50 66 43 69 59 42 61 40 71 51 65 53
 46 45 39 47 62 34 35 29 55 60 67 68 74 76 70 38 77]
Min = 29 Max = 77
--------------------------------------------------
sex
Count of uniqeu: 2
Values: [1 0]
Min = 0 Max = 1
--------------------------------------------------
cp
Count of uniqeu: 4
Values: [3 2 1 0]
Min = 0 Max = 3
--------------------------------------------------
trestbps
Count of uniqeu: 49
Values: [145 130 120 140 172 150 110 135 160 105 125 142 155 104 138 128 108 134
 122 115 118 100 124  94 112 102 152 101 132 148 178 129 180 136 126 106
 156 170 146 117 200 165 174 192 144 123 154 114 164]
Min = 94 Max = 200
--------------------------------------------------
chol
Count of uniqeu: 152
Values: [233 250 204 236 354 192 294 263 199 168 239 275 266 211 283 219 340 226
 247 234 243 302 212 175 417 197 198 177 273 213 304 232 269 360 308 245
 208 264 321 325 235 257 216 256 231 141 252 201 222 260 182 303 265 309
 1

Мы будем выполнять стандартизацию между числовыми переменными:  
* age (возраст): числовая переменная с возможным диапазоном от 20 до 80 лет
* trestbps (артериальное давление): числовая переменная, диапазон может быть от 90 до 200
* chol (уровень холестерина): числовая переменная, диапазон от 100 до 400
* thalach (максимальный пульс): числовая переменная, диапазон от 70 до 200  

sex, cp, fbs, restecg, exang — это категориальные или бинарные переменные, поэтому стандартизация им не нужна  

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_features = ['age', 'trestbps', 'chol', 'thalach']

X_scaled = X.copy()

X_scaled[numeric_features] = scaler.fit_transform(X_scaled[numeric_features])

X_scaled.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1


Проверим, что после стандартизации среднее стало близко к нулю, а среднеквадратичное отклонение приблизительно равно 1

In [20]:
np.mean(X_scaled['age']) < 0.0001

True

In [22]:
np.std(X_scaled['age'])

1.0

# Задача классификации

In [48]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Метрики для оценки качества классификатора

### Матрица путанницы, TP, FN, FP, TN

|                       | Предсказанный класс 1 | Предсказанный класс 2 |
|-----------------------|-----------------------|-----------------------|
| **Фактический класс 1** | True Positive (TP)       | False Negative (FN)     |
| **Фактический класс 2** | False Positive (FP)      | True Negative (TN)      |

True Positive (TP): Количество правильно классифицированных положительных объектов.  
$$
TP = \text{Количество объектов, правильно предсказанных как класс 1}
$$

False Negative (FN): Количество положительных объектов, неправильно классифицированных как отрицательные.  
$$
FN = \text{Количество объектов, фактически класс 1, но предсказанных как класс 2}
$$

False Positive (FP): Количество отрицательных объектов, неправильно классифицированных как положительные.  
$$
FP = \text{Количество объектов, фактически класс 2, но предсказанных как класс 1}
$$

True Negative (TN): Количество правильно классифицированных отрицательных объектов.  
$$
TN = \text{Количество объектов, правильно предсказанных как класс 2}
$$

### Точность, полнота и  среднее гармоническое

* Precision (точность):
$$
Precision =  \frac{TP}{TP + FP}
$$​  
Precision показывает, какая доля предсказанных положительных примеров (True Positives, TP) действительно является положительными, из всех, что модель предсказала как положительные. Высокая точность означает, что модель делает мало ложных срабатываний (False Positives, FP).  
Высокий precision важен, когда важно минимизировать ложные положительные срабатывания (например, при диагностике редких заболеваний).  

* Recall (полнота):
$$
Recall= \frac{TP}{TP + FN}
​$$  
Recall показывает, какая доля реальных положительных примеров была правильно предсказана моделью. То есть, из всех положительных примеров (True Positives + False Negatives, FN) — сколько модель распознала как положительные (TP).  
Высокий recall важен, когда нужно минимизировать количество пропущенных положительных случаев (например, пропущенные случаи мошенничества в финансовых транзакциях).  

* F1-score:
$$
F1_{score} = 2 \times \frac{Precision \times Recall}{Precision + Recall}
$$  
F1-score — это гармоническое среднее между precision и recall. F1 используется для баланса между precision и recall. Он полезен, когда важно учитывать как ложные срабатывания, так и пропуски.  
F1-score особенно полезен в задачах с несбалансированными классами, где один из классов встречается намного реже другого.  

* Support (поддержка):  

Support — это количество реальных наблюдений каждого класса в тестовом наборе. Например, если у вас два класса (0 и 1), support для класса 1 покажет, сколько примеров этого класса есть в реальных данных. Эта метрика помогает понять, как часто встречаются примеры каждого класса.

## Выбор моделей для классификации

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   age       303 non-null    int64
 1   sex       303 non-null    int64
 2   cp        303 non-null    int64
 3   trestbps  303 non-null    int64
 4   chol      303 non-null    int64
 5   fbs       303 non-null    int64
 6   restecg   303 non-null    int64
 7   thalach   303 non-null    int64
 8   exang     303 non-null    int64
 9   target    303 non-null    int64
dtypes: int64(10)
memory usage: 23.8 KB


In [ ]:
y.value_counts()

target
1    165
0    138
Name: count, dtype: int64

Для определения модели запишем основные характеристики датасета:  
* Размер датасета: 303 объекта, 10 признаков
* Цель классификации: Предсказание бинарного целевого признака (класс 0 или 1).
* Структура классов: Целевая переменная сбалансирована (165 объектов класса 1 и 138 объектов класса 0).
* Качество данных: Данные качественные без пропусков.

Выбранные модели:  

* Случайный лес  
Случайный лес — это ансамблевый метод, который работает путем создания множества деревьев решений и объединения их результатов. Может выявлять сложные закономерности в данных. Он устойчив к выбросам и может обрабатывать большое количество признаков.  

* Метод опорных векторов (SVM)
SVM эффективен для бинарной классификации и хорошо работает с высокоразмерными данными, особенно в случае, если данные разделимы с помощью сложной границы. Метод также хорошо подходит для задач, связанных с классификацией медицинских данных.

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

## Решающее дерево классификатор

In [45]:
param_grid_rf = {
    'n_estimators': [20, 50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

rf = RandomForestClassifier(random_state=42)

grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train_scaled, y_train)
rf_pred_scaled = grid_rf.predict(X_test_scaled)

print('Random forest best params:')
print(grid_rf.best_params_)
print(classification_report(y_test, rf_pred_scaled))

Random forest best params:
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       0.76      0.83      0.79        41
           1       0.85      0.78      0.81        50

    accuracy                           0.80        91
   macro avg       0.80      0.80      0.80        91
weighted avg       0.81      0.80      0.80        91



## Метод опорных векторов

In [46]:
param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

svc = SVC(random_state=42)

grid_svc = GridSearchCV(estimator=svc, param_grid=param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)
grid_svc.fit(X_train_scaled, y_train)

svc_pred_scaled = grid_svc.predict(X_test_scaled)

print('Support vector method best params:')
print(grid_rf.best_params_)
print(classification_report(y_test, svc_pred_scaled))

Support vector method best params:
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       0.82      0.76      0.78        41
           1       0.81      0.86      0.83        50

    accuracy                           0.81        91
   macro avg       0.81      0.81      0.81        91
weighted avg       0.81      0.81      0.81        91



# *Задача классификации без стандратизации данных

## Решающее дерево классификатор

In [49]:
rf = RandomForestClassifier(random_state=42)

grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)
rf_pred = grid_rf.predict(X_test_scaled)

print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.53      0.95      0.68        41
           1       0.89      0.32      0.47        50

    accuracy                           0.60        91
   macro avg       0.71      0.64      0.58        91
weighted avg       0.73      0.60      0.57        91



## Метод опорных векторов

In [50]:
svc = SVC(random_state=42)

grid_svc = GridSearchCV(estimator=svc, param_grid=param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)
grid_svc.fit(X_train, y_train)

svc_pred = grid_svc.predict(X_test)

print(classification_report(y_test, svc_pred))

              precision    recall  f1-score   support

           0       0.82      0.76      0.78        41
           1       0.81      0.86      0.83        50

    accuracy                           0.81        91
   macro avg       0.81      0.81      0.81        91
weighted avg       0.81      0.81      0.81        91



## Сравнение результатов

In [54]:
print(classification_report(y_test, rf_pred_scaled))
print('-' * 70)
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.76      0.83      0.79        41
           1       0.85      0.78      0.81        50

    accuracy                           0.80        91
   macro avg       0.80      0.80      0.80        91
weighted avg       0.81      0.80      0.80        91

----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.53      0.95      0.68        41
           1       0.89      0.32      0.47        50

    accuracy                           0.60        91
   macro avg       0.71      0.64      0.58        91
weighted avg       0.73      0.60      0.57        91



Как показала практика случайный лес сильно зависит от стандартизации данных, процент правильных ответов упал на 20, да и в целом метрики показывают качество ниже

In [55]:
print(classification_report(y_test, svc_pred_scaled))
print('-' * 70)
print(classification_report(y_test, svc_pred))

              precision    recall  f1-score   support

           0       0.82      0.76      0.78        41
           1       0.81      0.86      0.83        50

    accuracy                           0.81        91
   macro avg       0.81      0.81      0.81        91
weighted avg       0.81      0.81      0.81        91

----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.82      0.76      0.78        41
           1       0.81      0.86      0.83        50

    accuracy                           0.81        91
   macro avg       0.81      0.81      0.81        91
weighted avg       0.81      0.81      0.81        91



Метод опорных векторов показал большую устойчивость к нестандартизированным данным